# Analyze Portfolio via a standard process

- Gather stocks of interest
- Gather stocks from specific criteria (SP500 top 50...)
- Gather stocks from specific portfolio account
- Assemble stock universe 
- Gather price histories
- Compute aplha factors (X)
    - Compute custom apha factors
    - Compute univeral quant factors
    - Compute date information
- Compute target values (y)
- Generate Optimal Portfolio
- Report on new portfolio

In [1]:
from platform import python_version
import time
from datetime import datetime
import os
import pandas as pd
import numpy as np
import math
import matplotlib as plt

%matplotlib inline

import importlib
import trading_functions as tf
importlib.reload(tf)

portfun = tf.Portfolio()
selfun = tf.Selection()

import portfolio_optimizer
from portfolio_optimizer import OptimalHoldings
importlib.reload(portfolio_optimizer)

import ameritrade_functions as amc
importlib.reload(amc)

plt.rcParams['figure.figsize'] = (20, 8)
print(f'Python version: {python_version()}')
print(f'Pandas version: {pd.__version__}')

Python version: 3.6.12
Pandas version: 0.22.0


## Configure Ameritrade Information

Ameritrade credentials are stored in environment variables to keep from having unencrypted passwords stored on disk.

The module automatically masks the account numbers to protect the actual accounts. An Ameritrade user can have many investment accounts. We will be working with only one for this demonstration.

In [2]:
username = os.getenv('maiotradeuser')
password = os.getenv('maiotradepw')
client_id = os.getenv('maiotradeclientid')

masked_account_number = '#---5311'
account_portfolios_file_name = 'data/portfolio_data.csv'
portfolio_file_name = 'data/portfolio_' + masked_account_number[-4:] + '.csv'
price_histories_file_name = 'data/price_histories_20210424.csv'

## Authentication Tokens

To get data from Ameritrade you will need to obtains a short time use token (there is a re-use token, but that has not been coded yet.)

To obtain a token, you will need to have a Chrome driver located somewhere on your system. This will allow the module to use your credentials to obtain an authentication token.

In [3]:
td_ameritrade = amc.AmeritradeRest(username, password, client_id)
td_ameritrade.authenticate()

if len(td_ameritrade.authorization) == 0:
    print('Error: No authorization data: {}'.format(td_ameritrade.authorization))
else:
    print('You have authorization')

You have authorization


In [6]:
import trading_factors as alpha_factors
importlib.reload(alpha_factors)
import alphalens as al

def generate_portfolio(investment_amount, stock_universe, price_histories, risk_cap=0.05,weights_max=0.12, weights_min=-0.12, num_factor_exposures=10):
    print(f'Investment amount: {investment_amount}')
    pricing = datafun.get_close_values(price_histories)
    returns = retfun.compute_returns(pricing)
    
    all_factors = pd.concat(
    [
        alpha_factors.momentum(price_histories, 252),
        alpha_factors.mean_reversion_factor_returns_smoothed(price_histories, 5),
        alpha_factors.overnight_sentiment_smoothed(price_histories, 5)
    ], axis=1)
    all_factors.sort_index(inplace=True)

    assets = all_factors.index.levels[1].values.tolist()
    clean_factor_data, unixt_factor_data = alpha_factors.prepare_alpha_lense_factor_data(all_factors, pricing)
    
    
    """
    ls_factor_tears = pd.DataFrame()
    for factor, factor_data in clean_factor_data.items():
        print('Factor: ', factor)
        ls_factor_tears[factor] = al.tears.create_full_tear_sheet(factor_data, long_short=False, group_neutral=False, by_group=False)
        
    display(ls_factor_tears)
    """        
    risk_model = alpha_factors.RiskModelPCA(returns, 1, num_factor_exposures)
    print(f'portfolio variance is:  {risk_model.compute_portfolio_variance(sotck_universe_weights):.8f}')
    
    ls_factor_returns = pd.DataFrame()

    for factor, factor_data in clean_factor_data.items():
        ls_factor_returns[factor] = al.performance.factor_returns(factor_data).iloc[:, 0]
        
    (1+ls_factor_returns).cumprod().plot()
        
    qr_factor_returns = pd.DataFrame()

    for factor, factor_data in unixt_factor_data.items():
        qr_factor_returns[factor] = al.performance.mean_return_by_quantile(factor_data)[0].iloc[:, 0]

    (10000*qr_factor_returns).plot.bar(
        subplots=True,
        sharey=True,
        layout=(4,2),
        figsize=(14, 14),
        legend=False)
    
    ls_FRA = pd.DataFrame()

    for factor, factor_data in unixt_factor_data.items():
        ls_FRA[factor] = al.performance.factor_rank_autocorrelation(factor_data)

    ls_FRA.plot(title="Factor Rank Autocorrelation")
    
    factor_sharp_ratio = alpha_factors.sharpe_ratio(ls_factor_returns, 'daily')
    print(factor_sharp_ratio.round(2))
    
    selected_factors = all_factors.columns[[idx for idx, element in enumerate(factor_sharp_ratio.values) if element >= 0.9]]
    print(selected_factors)
    print('Selected Factors: {}'.format(', '.join(selected_factors)))

    all_factors['alpha_vector'] = all_factors[selected_factors].mean(axis=1)
    alphas = all_factors[['alpha_vector']]

    alpha_vector = alphas.loc[all_factors.index.get_level_values(0)[-1]]
    
    optimal_weights = OptimalHoldings(risk_cap=risk_cap,weights_max=weights_max, weights_min=weights_min).find(alpha_vector, risk_model.factor_betas_, risk_model.factor_cov_matrix_, risk_model.idiosyncratic_var_vector_)
    print(f'Old portfolio variance is:  {risk_model.compute_portfolio_variance(sotck_universe_weights):.8f}')
    print(f'New portfolio variance is:  {risk_model.compute_portfolio_variance(optimal_weights):.8f}')
    optimal_weights.plot.bar(legend=None, title='Portfolio % Holdings by Stock')
    
    print(f'Current investment amount: {investment_amount}')
    optimal_weights['amount'] = (optimal_weights['optimalWeights'] * investment_amount).round(0)
    optimal_weights['marketValue'] = stock_universe_values['marketValue']
    optimal_weights['buy/sell'] = (optimal_weights['marketValue'] - optimal_weights['amount']) * -1
    optimal_weights['close'] = pricing.iloc[-1]
    optimal_weights['deltaShares'] = (optimal_weights['buy/sell'] / optimal_weights['close']).round(0)
    optimal_weights['deltaMarketValue'] = (optimal_weights['deltaShares'] * optimal_weights['close'])
    optimal_weights['totalShares'] = optimal_weights['deltaShares']
    optimal_weights['totalMarketValue'] = (optimal_weights['totalShares'] * optimal_weights['close'])
    return optimal_weights[(optimal_weights['totalShares'] > 0) & (optimal_weights['deltaShares'] != 0)].round(2)

In [4]:
from trading_functions import Data

symbols_of_interest = ['MGM', 'PDYPF', 'NNXPF']
# Hardcoded for now
symbols_via_specific_criteria = ['AAPL', 'MSFT', 'GOOG', 'TSLA', 'COKE', 'IBM', 'BABA', 'GMGMF']

# Specific Portfolio Account
account_portfolio_df = Data().get_account_portfolio_data(td_ameritrade.parse_portfolios_list(), masked_account_number)
equity_investments_df = portfun.get_investments_by_type(account_portfolio_df, investment_type='EQUITY')
symbols_from_account = list(equity_investments_df['symbol'].values)

stock_universe = set(symbols_of_interest + symbols_via_specific_criteria + symbols_from_account)

In [5]:
non_portfolio_symbols = stock_universe - set(symbols_from_account)
non_portfolio_values = pd.DataFrame.from_dict({ symbol : [0, 0] for symbol in non_portfolio_symbols}, orient='index')
non_portfolio_values.index.name='symbol'
non_portfolio_values.columns = ['marketValue', 'longQuantity']
stock_universe_values = portfun.get_market_values(equity_investments_df).append(non_portfolio_values)
sotck_universe_weights = portfun.get_portfolio_weights(stock_universe_values)
display(sotck_universe_weights)

symbol
AAPL     0.000000
BABA     0.000000
COKE     0.000000
DKNG     0.020761
EEENF    0.000036
FGPHF    0.069914
FNCL     0.030159
GMGMF    0.000000
GOOG     0.000000
IBM      0.000000
MGM      0.000000
MSFT     0.000000
NNXPF    0.000000
PDYPF    0.000000
SCHA     0.017949
SCHF     0.077060
SCHM     0.053196
TLRY     0.108503
TSLA     0.000000
VBK      0.016120
VBR      0.019030
VGK      0.080243
VTI      0.313637
VXF      0.053188
WKHS     0.140203
dtype: float64

# Price History data

One you have a set of investments you want to work with, you will need to pull some historical data for them.

In [199]:
from trading_functions import Returns
from trading_functions import Data
number_of_years = 5
price_histories = td_ameritrade.get_price_histories(stock_universe, datetime.today().strftime('%Y-%m-%d'), num_periods=number_of_years)
returns = Returns().compute_returns(Data().get_close_values(price_histories))
pricing = Data().get_close_values(price_histories)

## Factors

In [200]:
import trading_factors as alpha_factors
importlib.reload(alpha_factors)
import alphalens as al

In [201]:
all_factors = pd.concat(
[
    alpha_factors.FactorMomentum(price_histories, 252).demean().rank().zscore().for_al(),
    alpha_factors.FactorMeanReversion(price_histories, 5).demean().rank().zscore().smoothed().rank().zscore().for_al(),
    alpha_factors.OvernightSentiment(price_histories, 5).demean().rank().zscore().smoothed(10).rank().zscore().for_al()
], axis=1).dropna()
all_factors.sort_index(inplace=True)

## Universal Quant Features

- Volatility 
- Dollar Volume
- Market Dispersion
- Market Volatility

In [202]:
all_factors = pd.concat(
[
    all_factors,
    alpha_factors.AnnualizedVolatility(price_histories, 20).rank().zscore().for_al(),
    alpha_factors.AnnualizedVolatility(price_histories, 120).rank().zscore().for_al(),
    alpha_factors.AverageDollarVolume(price_histories, 20).rank().zscore().for_al(),
    alpha_factors.AverageDollarVolume(price_histories, 120).rank().zscore().for_al(),
    alpha_factors.MarketDispersion(price_histories, 20).for_al(),
    alpha_factors.MarketDispersion(price_histories, 120).for_al(),
    alpha_factors.MarketVolatility(price_histories, 20).for_al(),
    alpha_factors.MarketVolatility(price_histories, 120).for_al()
], axis=1).dropna()
all_factors.sort_index(inplace=True)

In [203]:
alpha_factors.FactorDateParts(all_factors)
all_factors.head()

momentum_252_day_logret  \
date       ticker                            
2017-05-31 AAPL                   1.535798   
           BABA                   1.256562   
           COKE                   1.675416   
           DKNG                  -0.907517   
           EEENF                 -0.907517   

                   mean_reversion_5_day_logret_smoothed  \
date       ticker                                         
2017-05-31 AAPL                               -0.837708   
           BABA                               -1.535798   
           COKE                               -0.139618   
           DKNG                                0.488663   
           EEENF                               0.488663   

                   overnight_sentiment_5_day_smoothed  \
date       ticker                                       
2017-05-31 AAPL                              1.387818   
           BABA                              0.971473   
           COKE                              1.526600   
           DKNG                             -1.665382   
           EEENF                            -0.555127   

                   annualzed_volatility_20_day  annualzed_volatility_120_day  \
date       ticker                                                              
2017-05-31 AAPL                       0.977326                      0.279236   
           BABA                      -0.139618                      1.256562   
           COKE                       1.535798                      1.396180   
           DKNG                      -1.326371                     -1.326371   
           EEENF                     -1.326371                     -1.326371   

                   average_dollar_volume_20_day  \
date       ticker                                 
2017-05-31 AAPL                        1.682316   
           BABA                        1.261737   
           COKE                       -0.280386   
           DKNG                       -1.261737   
           EEENF                      -1.261737   

                   average_dollar_volume_120_day  market_dispersion20_day  \
date       ticker                                                           
2017-05-31 AAPL                         1.670538                 0.016915   
           BABA                         1.113692                 0.016915   
           COKE                        -0.556846                 0.016915   
           DKNG                        -1.392115                 0.016915   
           EEENF                       -0.974481                 0.016915   

                   market_dispersion120_day  market_volatility20_day  \
date       ticker                                                      
2017-05-31 AAPL                     0.01239                 0.072422   
           BABA                     0.01239                 0.072422   
           COKE                     0.01239                 0.072422   
           DKNG                     0.01239                 0.072422   
           EEENF                    0.01239                 0.072422   

                   market_volatility120_day  is_January  is_December  weekday  \
date       ticker                                                               
2017-05-31 AAPL                    0.058272           0            0        2   
           BABA                    0.058272           0            0        2   
           COKE                    0.058272           0            0        2   
           DKNG                    0.058272           0            0        2   
           EEENF                   0.058272           0            0        2   

                   quarter  year  month_start  month_end  quarter_start  \
date       ticker                                                         
2017-05-31 AAPL          2  2017            0          1              0   
           BABA          2  2017            0          1              0   
           COKE          2  2017            0     

## Compute the target values (y)

In [204]:
alpha_factors.FactorReturnQuantiles(price_histories, 5, 5).factor_data.head()

all_factors = pd.concat(
[
    all_factors,
    alpha_factors.FactorReturnQuantiles(price_histories, 5, 5).factor_data
], axis=1).dropna()
all_factors.sort_index(inplace=True)
all_factors.head()

momentum_252_day_logret  \
date       ticker                            
2017-05-31 AAPL                   1.535798   
           BABA                   1.256562   
           COKE                   1.675416   
           DKNG                  -0.907517   
           EEENF                 -0.907517   

                   mean_reversion_5_day_logret_smoothed  \
date       ticker                                         
2017-05-31 AAPL                               -0.837708   
           BABA                               -1.535798   
           COKE                               -0.139618   
           DKNG                                0.488663   
           EEENF                               0.488663   

                   overnight_sentiment_5_day_smoothed  \
date       ticker                                       
2017-05-31 AAPL                              1.387818   
           BABA                              0.971473   
           COKE                              1.526600   
           DKNG                             -1.665382   
           EEENF                            -0.555127   

                   annualzed_volatility_20_day  annualzed_volatility_120_day  \
date       ticker                                                              
2017-05-31 AAPL                       0.977326                      0.279236   
           BABA                      -0.139618                      1.256562   
           COKE                       1.535798                      1.396180   
           DKNG                      -1.326371                     -1.326371   
           EEENF                     -1.326371                     -1.326371   

                   average_dollar_volume_20_day  \
date       ticker                                 
2017-05-31 AAPL                        1.682316   
           BABA                        1.261737   
           COKE                       -0.280386   
           DKNG                       -1.261737   
           EEENF                      -1.261737   

                   average_dollar_volume_120_day  market_dispersion20_day  \
date       ticker                                                           
2017-05-31 AAPL                         1.670538                 0.016915   
           BABA                         1.113692                 0.016915   
           COKE                        -0.556846                 0.016915   
           DKNG                        -1.392115                 0.016915   
           EEENF                       -0.974481                 0.016915   

                   market_dispersion120_day  market_volatility20_day  \
date       ticker                                                      
2017-05-31 AAPL                     0.01239                 0.072422   
           BABA                     0.01239                 0.072422   
           COKE                     0.01239                 0.072422   
           DKNG                     0.01239                 0.072422   
           EEENF                    0.01239                 0.072422   

                             ...             is_January  is_December  weekday  \
date       ticker            ...                                                
2017-05-31 AAPL              ...                    0.0          0.0      2.0   
           BABA              ...                    0.0          0.0      2.0   
           COKE              ...                    0.0          0.0      2.0   
           DKNG              ...                    0.0          0.0      2.0   
           EEENF             ...                    0.0          0.0      2.0   

                   quarter    year  month_start  month_end  quarter_start  \
date       ticker                                                           
2017-05-31 AAPL        2.0  2017.0          0.0        1.0            0.0   
           BABA        2.0  2017.0          0.0        1.0            0.0   
           COKE        2.0  2017.0        

### View Data
With the `OptimalHoldings` class implemented, let's see the weights it generates.

In [ ]:
optimal_weights = OptimalHoldings(risk_cap=0.05,weights_max=0.12, weights_min=-0.12).find(alpha_vector, risk_model.factor_betas_, risk_model.factor_cov_matrix_, risk_model.idiosyncratic_var_vector_)
print(f'Old portfolio variance is:  {risk_model.compute_portfolio_variance(sotck_universe_weights):.8f}')
print(f'New portfolio variance is:  {risk_model.compute_portfolio_variance(optimal_weights):.8f}')
display((optimal_weights*100).round(2))
optimal_weights.plot.bar(legend=None, title='Portfolio % Holdings by Stock')

#x_axis = plt.axes().get_xaxis()
#x_axis.set_visible(False)

In [ ]:
investment_amount = portfun.get_account_value(stock_universe_values).round(2)
print(f'Current investment amount: {investment_amount}')
optimal_weights['amount'] = (optimal_weights['optimalWeights'] * investment_amount).round(0)
optimal_weights['marketValue'] = stock_universe_values['marketValue']
optimal_weights['buy/sell'] = (optimal_weights['marketValue'] - optimal_weights['amount']) * -1
optimal_weights['close'] = pricing.iloc[-1]
optimal_weights['existingShares'] = stock_universe_values['longQuantity']
optimal_weights['deltaShares'] = (optimal_weights['buy/sell'] / optimal_weights['close']).round(0)
optimal_weights['deltaMarketValue'] = (optimal_weights['deltaShares'] * optimal_weights['close'])
optimal_weights['totalShares'] = (optimal_weights['existingShares'] + optimal_weights['deltaShares'])
optimal_weights['totalMarketValue'] = (optimal_weights['totalShares'] * optimal_weights['close'])
optimal_weights

## Think or Swim Simulator

In [ ]:
optimal_weights[(optimal_weights['totalShares'] > 0) | (optimal_weights['deltaShares'] != 0)].round(2)

In [ ]:
optimal_weights[(optimal_weights['totalShares'] <= 0) & (optimal_weights['deltaShares'] == 0)].round(2)

In [ ]:
optimal_weights[(optimal_weights['totalShares'] > 0) | (optimal_weights['deltaShares'] != 0)][['deltaMarketValue', 'totalMarketValue']].sum().round(2)